# Handwritten digit recognition using pytorch

## Import dependency

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


## Load dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load MNIST dataset
train_dataset = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)




## Build Model


In [ ]:
class DigitRecognitionModel(nn.Module):
  def __init__(self):
    super(DigitRecognitionModel, self).__init__()

    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0)

    self.fc1 = nn.Linear(64 * 3 * 3, 64)
    self.fc2 = nn.Linear(64, 10)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.pool1(x)
    x = self.relu(self.conv2(x))
    x = self.pool2(x)
    x = self.relu(self.conv3(x))

    x = x.view(x.size(0), -1)

    x = self.relu(self.fc1(x))
    x = self.fc2(x)

    return x

## Initialize model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DigitRecognitionModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


## Training function

In [ ]:
def train(mode, train_loader, optimiser, criterion, device):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  avg_loss = running_loss / len(train_loader)

  return avg_loss, accuracy


## Evaluation function

In [ ]:
def evaluate(model, test_loader, criterion, device):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0

  with torch.no_grad():
    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      running_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  accuracy = 100 * correct / total
  avg_loss = running_loss / len(test_loader)

  return avg_loss, accuracy


## Training

In [ ]:
epochs = 5

for epoch in range(epochs):
  train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
  test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)

  print(f"Epoch {epoch +1}/{epochs}")
  print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}")
  print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}")
  print("-" * 60)


# Final evaluation
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}")



## Prediction

In [ ]:
num_examples = 5

# Get random smaples
example_idx = np.random.choice(len(test_dataset), num_examples)

plt.figure(figsize=(12, 6))

for i, idx in enumerate(example_idx):
  image, true_label = test_dataset[idx]

  # Predict
  with torch.no_grad():
    image = image.unsqueeze(0).to(device)
    output = model(image)
    probabilities = torch.nn.functional.softmax(output, dim=1)[0]
    predicted_label = torch.argmax(probabilities).item()
    confidence = probabilities[predicted_label].item() * 100

  plt.subplot(1, num_examples, i+1)
  plt.imshow(image.cpu().squeeze().numpy(), cmap="gray")
  plt.title(f"True: {true_label}\nPredicted: {predicted_label}\nConfidence: {confidence:.2f}%")
  plt.axis("off")

plt.tight_layout()
plt.show()

## Save model

In [ ]:
torch.save(model.state_dict(), 'digit_recognition_model.pth')